# ROS integration

rigid_body_motion supports certain ROS functionality, provided the Python environment has been set up with the required ROS packages. This guide assumes that you are at least somewhat familiar with ROS concepts such as nodes, publishers/subscribers and messages. If not, the [ROS tutorials](http://wiki.ros.org/ROS/Tutorials) are a good place to start.

You also need to set up a couple of dependencies which can be done very conveniently if you are using an Anaconda Python distribution. See [the ROS dependencies installation guide](installation.rst#ros-install) for further information.

<div class="alert alert-info">
Note
    
The following examples require the `pooch`, `xarray`, `netcdf4` and `ipywidgets` libraries.
</div>

In [1]:
import numpy as np
import rigid_body_motion as rbm
import rospy
import xarray as xr

## Loading data from rosbag files

Data can be loaded from rosbag files into numpy arrays. So far, `geometry_msgs/TransformStamped` and `nav_msgs/Odometry` messages are supported. This is done through the `RosbagReader` class:

In [2]:
reader = rbm.ros.RosbagReader(rbm.example_data["rosbag"])

The reader can be used as a context manager to facilitate opening and closing of the rosbag. The `get_topics_and_types` method returns a dict with topic names and corresponding message types:

In [3]:
with reader:
    info = reader.get_topics_and_types()
info

{'/pupil/left_eye/transform': 'geometry_msgs/TransformStamped',
 '/pupil/right_eye/transform': 'geometry_msgs/TransformStamped',
 '/t265/transform': 'geometry_msgs/TransformStamped'}

The data included in the example rosbag is from a [head/eye tracking study](https://dl.acm.org/doi/pdf/10.1145/3379156.3391365) and contains head-in-world pose estimated by the Intel RealSense T265 as well as eye-in-head pose for both eyes estimated by the Pupil Core eye tracker. 

The `load_messages` method returns a dict with the data from a specified topic:

In [4]:
with reader:
    head = reader.load_messages("/t265/transform")
head

{'timestamps': array([1.58060323e+09, 1.58060323e+09, 1.58060323e+09, ...,
        1.58060357e+09, 1.58060357e+09, 1.58060357e+09]),
 'position': array([[15.9316,  0.8211, 10.5429],
        [15.9354,  0.8208, 10.5382],
        [15.9393,  0.8204, 10.5335],
        ...,
        [29.8883,  2.8952,  7.6317],
        [29.8888,  2.8943,  7.6249],
        [29.8892,  2.8935,  7.6182]]),
 'orientation': array([[-0.9687,  0.0917,  0.2306,  0.0039],
        [-0.969 ,  0.0915,  0.2295,  0.005 ],
        [-0.9693,  0.0912,  0.2285,  0.0061],
        ...,
        [-0.9915,  0.0915,  0.0929, -0.0022],
        [-0.9914,  0.0922,  0.0927, -0.0017],
        [-0.9913,  0.0932,  0.0925, -0.001 ]])}

Now we can construct a reference frame tree with this data:

In [5]:
rbm.register_frame("world")

In [6]:
R_T265_ROS = np.array([[0.0, 0.0, -1.0], [-1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])

rbm.ReferenceFrame.from_rotation_matrix(
    R_T265_ROS, parent="world", name="t265/world"
).register()

In [7]:
rbm.register_frame(
    "t265/tracker",
    parent="t265/world",
    translation=head["position"],
    rotation=head["orientation"],
    timestamps=head["timestamps"],
)

In [8]:
rbm.ReferenceFrame.from_rotation_matrix(
    R_T265_ROS, parent="t265/tracker", name="head", inverse=True,
).register()

In [9]:
rbm.render_tree("world")

world
└── t265/world
    └── t265/tracker
        └── head


## Visualization with RViz

You can download an `.rviz` file where all topics created in the following are already set up [here](_static/example.rviz).

In [10]:
rospy.init_node("rbm_vis")

In [11]:
tf_world_head = rbm.ros.ReferenceFrameTransformBroadcaster("head", base="world")

In [12]:
tf_world_head.publish()

In [13]:
marker_publisher = rbm.ros.ReferenceFrameMarkerPublisher("head", base="world")

In [14]:
marker_publisher.publish()

<img src="_static/img/rviz_1.png" style="width: 800px;"/>

In [15]:
with reader:
    left_eye = reader.load_dataset("/pupil/left_eye/transform", cache=True)

In [16]:
t_t265_pupil = (24.5e-3, -29e-3, 0.0)
r_t265_pupil = (-0.00125, -1.0, 6.3463e-05, 3.977e-06)

rbm.register_frame(
    "pupil/scene_cam",
    parent="t265/tracker",
    translation=t_t265_pupil,
    rotation=r_t265_pupil,
)

In [17]:
rbm.ReferenceFrame.from_dataset(
    left_eye,
    "position",
    "orientation",
    "time",
    parent="pupil/scene_cam",
    name="pupil/left_eye",
).register()

In [18]:
R_PUPIL_ROS = np.array([[0.0, -1.0, 0.0], [0.0, 0.0, -1.0], [1.0, 0.0, 0.0]])

rbm.ReferenceFrame.from_rotation_matrix(
    R_PUPIL_ROS, parent="pupil/left_eye", name="left_eye"
).register(update=True)

In [19]:
rbm.render_tree("world")

world
└── t265/world
    └── t265/tracker
        ├── head
        └── pupil/scene_cam
            └── pupil/left_eye
                └── left_eye


In [20]:
tf_head_left_eye = rbm.ros.ReferenceFrameTransformBroadcaster(
    "left_eye", base="head", publish_pose=True, subscribe=True
)

In [21]:
tf_head_left_eye.spin()

In [22]:
rbm.ros.play_publisher(tf_world_head, step=2, skip=1, speed=0.5)

Output()

<img src="_static/img/rviz_2.png" style="width: 800px;"/>